# Agnostic methods for liquid handling


## Setting up 

Check if the correct environment is in use:

In [ ]:
!echo $VIRTUAL_ENV

import os, sys
print("Current working directory: " + os.getcwd())
print("Using python: " + sys.executable)

?

In [ ]:
%load_ext autoreload
%autoreload 2

Basic imports:

In [ ]:
# Basic imports:
# - liquid handler
from pylabrobot.liquid_handling import LiquidHandler
# - backend
from pylabrobot.liquid_handling.backends import STAR
# - deck
from pylabrobot.resources.hamilton import STARLetDeck

## Writing a custom deck

In [ ]:
import textwrap
from typing import Optional, Callable

from pylabrobot.resources import Coordinate, Deck, Trash


class SilverDeck(Deck):
  """ (Ag)nostic deck object.
  
  Boilerplate code written by Rick: https://forums.pylabrobot.org/t/writing-a-new-backend-agnosticity/844/16
  """

  def __init__(self, 
               name: str= "silver_deck",
               # TODO: Update default size.
               size_x: float = 250,
               size_y: float = 350,
               size_z: float = 200,
               resource_assigned_callback: Optional[Callable] = None,
               resource_unassigned_callback: Optional[Callable] = None,
               # TODO: Update default origin.
               origin: Coordinate = Coordinate(0, 0, 0),
               # TODO: Update default trash location.
               trash_location: Coordinate = Coordinate(x=82.84, y=53.56, z=5),
               no_trash: bool = False):

    # Run init from the base Deck class.
    super().__init__(
      name=name,
      size_x=size_x, size_y=size_y, size_z=size_z,
      resource_assigned_callback=resource_assigned_callback,
      resource_unassigned_callback=resource_unassigned_callback,
      origin=origin)
    
    # TODO: write your init code, for example assign a "trash" resource:
    if not no_trash:
      self._assign_trash(location=trash_location)

  def _assign_trash(self, location: Coordinate):
    """ Assign the trash area to the deck. """

    trash = Trash(
      name="trash",
      # TODO: Update default dimensions.
      size_x=80,
      size_y=120,
      size_z=50
    )

    self.assign_child_resource(trash, location=location)

  def summary(self) -> str:
    """ Get a summary of the deck.

    >>> print(deck.summary())

    TODO: <write some printable ascii representation of the deck's current layout>
    """

    return textwrap.dedent(f"""\
      +---------------------+
      |                     |
      |        TODO         |
      |                     |
      +---------------------+
    """)


Instantiate a new deck object:

In [ ]:
deck = SilverDeck(name="basic deck 1")

Stuff can be added to the deck, which in PLR jargon is the same as "assigning a child resource" to the deck.

This is typically accomplished by calling `assign_child_resource`, a method from the Resource class,
which is also the baseclass for Deck.

This means that our deck has a list of resources, which are considered children of the deck resource:

In [ ]:
deck.children

Lets add another resource, a well plate:

In [ ]:
# Import the resource class
from pylabrobot.resources import Cos_96_DW_1mL

# Create an instance
well_plate = Cos_96_DW_1mL(name='plate_01')

Note that the well plate is also a resource, and also has children. Lets print the first five children, and realize that it has been populated with "Well" type resources:

In [ ]:
well_plate.children[:5]

Now we will assign the well plate resource as a child of our deck resource using `assign_child_resource`.

The `assign_child_resource` function inherited from base `Resource` class accepts an optional `location` parameter, with type `Coordinate` (another type of PLR object). If passed, it is used to override the `location` attribute of the Resource we want to add.

 Lets have a look:

In [ ]:
# Note that the well has no default location.
print(well_plate.location)

In [ ]:
# Since some robots have "slots" or "rails", the syntax will vary.
# For example, the pyhamilton deck has overriden the "assign_child_resource"
# method from "Resource", such that it accepts a "rail" parameter instead 
# of a location/Coordinate.
deck.assign_child_resource(well_plate, location=Coordinate(10, 10, 0))

In [ ]:
# Note that the well now has location.
print(well_plate.location)

In [ ]:
# Note that the well plate now has a parent
print(well_plate.parent)

In [ ]:
# Note that the deck now has an additional child.
deck.children

Lets try to break PLR by assigning the same object again:

In [ ]:
try:
  deck.assign_child_resource(well_plate, location=Coordinate(10, 10, 0), reassign=False)
except Exception as e:
  print(f"Error: {e}")
else:
  print(f"No error :/")

Lets try to break PLR by re-assigning the same child to another parent, with the reassing disabled:

In [ ]:
try:
  deck2 = SilverDeck(name="second deck")
  deck2.assign_child_resource(well_plate, location=Coordinate(10, 10, 0), reassign=False)
except Exception as e:
  print(f"Error: {e}")
else:
  print(f"No error :/")

Note that `reassing=True` (the default) actually reassigns the resource to a different parent.

In [ ]:
try:
  deck2 = SilverDeck(name="second deck")
  deck2.assign_child_resource(well_plate, location=Coordinate(10, 10, 0))
except Exception as e:
  print(f"Error: {e}")


In [ ]:
well_plate.parent.name

Lets try to break PLR by assigning a Resource to itself:

In [ ]:
try:
  well_plate.assign_child_resource(well_plate, location=Coordinate(10, 10, 0))
except Exception as e:
  print(f"Error: {e}")
else:
  print(f"No error :/")

## STAR deck examples


Create a new liquid handler using `STAR` as its backend, and the hamilton "deck".

In [ ]:
backend = STAR()
lh = LiquidHandler(backend=backend, deck=STARLetDeck())

The final step is to open communication with the robot. This is done using the {func}`~pylabrobot.liquid_handling.LiquidHandler.setup` method.

In [ ]:
await lh.setup()


Now that we have a `LiquidHandler` instance, we can define the deck layout.

The layout in this tutorial will contain five sets of standard volume tips with filter, 1 set of 96 1mL wells, and tip and plate carriers on which these resources are positioned.

Start by importing the relevant objects and variables from the PyHamilton package. This notebook uses the following resources:

- {class}`~pylabrobot.resources.ml_star.tip_carriers.TIP_CAR_480_A00` tip carrier
- {class}`~pylabrobot.resources.ml_star.plate_carriers.PLT_CAR_L5AC_A00` plate carrier
- {class}`~pylabrobot.resources.corning_costar.plates.Cos_96_DW_1mL` wells
- {class}`~pylabrobot.resources.ml_star.tip_racks.HTF_L` tips

In [ ]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    HTF_L
)

Then create a tip carrier named `tip carrier`, which will contain tip rack at all 5 positions. These positions can be accessed using `tip_car[x]`, and are 0 indexed.

In [ ]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = HTF_L(name='tips_01')

Use {func}`~pylabrobot.resources.abstract.assign_child_resources` to assign the tip carrier to the deck of the liquid handler. All resources contained by this carrier will be assigned automatically.

In the `rails` parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.

In [ ]:
lh.deck.assign_child_resource(tip_car, rails=3)

Repeat this for the plates.

In [ ]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='plate_01')

In [ ]:
lh.deck.assign_child_resource(plt_car, rails=15)

Let's look at a summary of the deck layout using {func}`~pylabrobot.liquid_handling.LiquidHandler.summary`.

In [ ]:
lh.summary()

## Picking up tips

Picking up tips is as easy as querying the tips from the tiprack.

In [ ]:
tiprack = lh.get_resource("tips_01")
await lh.pick_up_tips(tiprack["A1:C1"])

## Aspirating and dispensing

Aspirating and dispensing work similarly to picking up tips: where you use booleans to specify which tips to pick up, with aspiration and dispensing you use floats to specify the volume to aspirate or dispense in $\mu L$.

The cells below move liquid from wells `'A1:C1'` to `'D1:F1'` using channels 1, 2, and 3 using the {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate` and {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense` methods.

In [ ]:
plate = lh.get_resource("plate_01")
await lh.aspirate(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

After the liquid has been aspirated, dispense it in the wells below. Note that while we specify different wells, we are still using the same channels. This is needed because only these channels contain liquid, of course.

In [ ]:
await lh.dispense(plate["D1:F1"], vols=[100.0, 50.0, 200.0])

Let's move the liquid back to the original wells.

In [ ]:
await lh.aspirate(plate["D1:F1"], vols=[100.0, 50.0, 200.0])
await lh.dispense(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

## Discarding tips

Finally, you can discard tips by using the {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips` method.

In [ ]:
await lh.drop_tips(tiprack["A1:C1"])

In [ ]:
await lh.stop()